# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd
import os

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
os.chdir("../files")
publications = pd.read_excel("Publications.xlsx", dtype=str)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2009-07-01 00:00:00,Ground-based Pc5 ULF wave power: Solar wind sp...,Journal of Atmospheric and Solar-Terrestrial P...,NaN,"Pahud, D. M., Rae, I. J., Mann, I. R., Murphy,...",Pahud,https://doi.org/10.1016/j.jastp.2008.12.004
1,2008-12-01 00:00:00,The Upgraded CARISMA Magnetometer Array in the...,Space Science Reviews,NaN,"Mann, I. R., Milling, D. K., Rae, I. J., Ozeke...",Mann,https://doi.org/10.1007/s11214-008-9457-6
2,2008-07-01 00:00:00,Ionospheric localisation and expansion of long...,Geophysical Research Letters,NaN,"Milling, D. K., Rae, I. J., Mann, I. R., Murph...",Milling,https://doi.org/10.1029/2008GL033672
3,2009-12-01 00:00:00,Timing and localization of near-Earth tail and...,Journal of Geophysical Research: Space Physics,NaN,"Gabrielse, C., Angelopoulos, V., Runov, a., Fr...",Gabrielse,https://doi.org/10.1029/2008JA013583
4,2009-05-01 00:00:00,THEMIS observation of a substorm event on 04:3...,Annales Geophysicae,NaN,"Liu, J., Angelopoulos, V., Frey, H., McFadden,...",Liu,https://doi.org/10.5194/angeo-27-1831-2009
...,...,...,...,...,...,...,...
101,2024-09-26 00:00:00,Target and Science Visibility of the Solar-Ter...,Frontiers in Astronomy and Space Sciences,NaN,NaN,Murphy,https://doi.org/10.3389/fspas.2024.1394655
102,2024-11-27 00:00:00,Diamond Experiment In the MagnetOSphere (DEIMO...,Journal of Geophysical Research: Space Physics,NaN,"Shonibare, T., Murphy, K. R., & Pakhotin, I. P...",Shonibare,https://doi.org/10.1029/2024JA032728
103,2024-11-27 00:00:00,Derivations of the Total Radiation Belt Electr...,Journal of Geophysical Research: Space Physics,NaN,"Pitzel, J. C., Cully, C. M., Ripoll, J.-F., Lo...",Pitzel,https://doi.org/10.1029/2024JA032940
104,2025-01-06 00:00:00,Understanding and Modeling the Dynamics of Sto...,Space Weather,NaN,"Murphy, K., Halford, A. J., Liu, V., Klenzing,...",Murphy,https://doi.org/10.1029/2024SW003928


In [15]:
publications.iloc[101]['citation'].type

AttributeError: 'float' object has no attribute 'type'

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [19]:
os.chdir("../_publications")
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date[0:10]) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date[0:10]) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    # add an empty excerpt so that the html generator does't
    # grab text from outside the file header
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    else:
        md += "\nexcerpt: ' '"
    md += "\ndate: " + str(item.pub_date[0:10]) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    if len(str(item.citation)) >5:
        md += "\ncitation: '" + html_escape(item.citation) + "'"
    else:
        md += "\ncitation: ' '"
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    #if len(str(item.paper_url)) > 5:
        #md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    #md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open(md_filename, 'w', encoding="utf-8") as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).